In [11]:
# Import libraries and load the dataset

import pandas as pd
import numpy as np

# Load dataset
data = pd.read_csv("heart-disease-UCI.csv")

# Display first few rows
data.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [12]:
# Basic information and stats
data.info()
data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [13]:
# Explore target variable distribution
data['target'].value_counts()




target
1    165
0    138
Name: count, dtype: int64

In [14]:
# Check for missing values
data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [15]:
# Handle Missing Data (Numeric + Categorical, Proximity-based)
from sklearn.impute import KNNImputer

# Make a copy of the data to work safely
df = data.copy()

# Step 1 — Drop columns with excessive missingness (>50%)
df = df.drop(columns=['ca', 'thal', 'slope'])  # too many NaNs

# Step 2 — KNN Imputation for numeric features
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Initialize imputer
knn_imputer = KNNImputer(n_neighbors=5)

# Apply KNN imputation
df_numeric_imputed = pd.DataFrame(knn_imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)

# Replace numeric columns with imputed values
df[numeric_cols] = df_numeric_imputed

# Proximity-based + fallback imputation for categorical columns
def fill_categorical_mode_with_fallback(df, target_col, group_cols):
    # Group-based fill using mode (proximity)
    df[target_col] = df.groupby(group_cols)[target_col].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x
    )
    # Fallback global mode if still missing
    if df[target_col].isnull().any():
        global_mode = df[target_col].mode().iloc[0]
        df[target_col].fillna(global_mode, inplace=True)
    return df

# Handle remaining categorical columns manually
# These are known categorical / binary columns in Heart dataset
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang']

# Apply proximity-mode imputation
df = fill_categorical_mode_with_fallback(df, 'fbs', ['sex'])
df = fill_categorical_mode_with_fallback(df, 'restecg', ['sex'])
df = fill_categorical_mode_with_fallback(df, 'exang', ['sex', 'cp'])

# Verify all missing values handled
print("Missing values per column after cleaning:\n")
print(df.isnull().sum())

# Final clean dataset
data_cleaned = df.copy()




Missing values per column after cleaning:

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
target      0
dtype: int64


In [16]:
# Perform one-hot encoding for categorical variables
data_encoded = pd.get_dummies(data_cleaned, columns=categorical_cols, drop_first=True)

# Display new shape and head
data_encoded.shape, data_encoded.head()


((303, 14),
     age  trestbps   chol  thalach  oldpeak  target  sex_1.0  cp_1.0  cp_2.0  \
 0  63.0     145.0  233.0    150.0      2.3     1.0     True   False   False   
 1  37.0     130.0  250.0    187.0      3.5     1.0     True   False    True   
 2  41.0     130.0  204.0    172.0      1.4     1.0    False    True   False   
 3  56.0     120.0  236.0    178.0      0.8     1.0     True    True   False   
 4  57.0     120.0  354.0    163.0      0.6     1.0    False   False   False   
 
    cp_3.0  fbs_1.0  restecg_1.0  restecg_2.0  exang_1.0  
 0    True     True        False        False      False  
 1   False    False         True        False      False  
 2   False    False        False        False      False  
 3   False    False         True        False      False  
 4   False    False         True        False       True  )